# Networking (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

Installing Packages Expecto...

Installing Packages Expecto....

Installing Packages Expecto.....

Installing Packages Expecto......

Installing Packages Expecto.......

Installing Packages Expecto........

Installed Packages Expecto, 10.1.0

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs

In [ ]:
open Common

## testPortOpen

In [ ]:
let testPortOpen port = async {
    use client = new System.Net.Sockets.TcpClient ()
    try
        do! client.ConnectAsync ("127.0.0.1", port) |> Async.AwaitTask
        return true
    with ex ->
        trace Warn (fun () -> $"testPortOpen / message: {ex.Message}") getLocals
        return false
}

In [ ]:
//// test

testPortOpen 65536
|> Async.RunSynchronously
|> _equal false

03:19:52 #1 [Warn] testPortOpen / message: Specified argument was out of the range of valid values. (Parameter 'port')
False


## waitForPortAccess

In [ ]:
let waitForPortAccess status port =
    let rec loop retry = async {
        let! isPortOpen = testPortOpen port
        if isPortOpen = status
        then return retry
        else
            if retry % 100 = 0 then
                let getLocals () = $"port: {port} / retry: {retry} / {getLocals ()}"
                trace Warn (fun () -> "waitForPortAccess") getLocals
            do! Async.Sleep 1
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let port = 5000

let lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 5000
    let listener = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! _ = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries = waitForPortAccess true port
    retries |> _isGreaterThanOrEqual 2
    let! retries = waitForPortAccess false port
    retries |> _isGreaterThanOrEqual 100
}
|> Async.runWithTimeout 15000
|> _equal (Some ())

03:19:53 #2 [Debug] 1
03:19:53 #4 [Debug] 2
03:19:53 #3 [Debug] _1
03:19:53 #5 [Debug] 3
03:19:55 #6 [Warn] testPortOpen / message: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
03:19:55 #7 [Warn] waitForPortAccess / port: 5000 / retry: 0
03:19:57 #8 [Warn] testPortOpen / message: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
03:19:58 #9 [Debug] _2
03:19:58 #10 [Debug] _3
2
03:19:58 #11 [Warn] waitForPortAccess / port: 5000 / retry: 0
03:19:59 #12 [Warn] waitForPortAccess / port: 5000 / retry: 100
03:20:00 #13 [Debug] _4
03:20:00 #14 [Debug] _5
03:20:02 #15 [Warn] testPortOpen / message: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
122
FSharpOption<Unit>
      Value: <null>


## getAvailablePort

In [ ]:
let getAvailablePort initialPort =
    let rec loop port = async {
        let! isPortOpen = testPortOpen port
        if not isPortOpen
        then return port
        else return! loop (port + 1)
    }
    loop initialPort

In [ ]:
//// test

let port = 5000

let lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    let listener1 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    let listener2 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 4000
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! _ = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort port
    availablePort |> _equal (port + 2)
    let! retries = waitForPortAccess false port
    retries |> _isGreaterThanOrEqual 100
}
|> Async.runWithTimeout 10000
|> _equal (Some ())

03:20:02 #16 [Debug] 1
03:20:02 #17 [Debug] 2
03:20:02 #18 [Debug] _1
03:20:02 #19 [Debug] _2
03:20:02 #20 [Debug] _3
03:20:02 #21 [Debug] 3
03:20:04 #22 [Warn] testPortOpen / message: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
5002
03:20:04 #23 [Warn] waitForPortAccess / port: 5000 / retry: 0
03:20:05 #24 [Warn] waitForPortAccess / port: 5000 / retry: 100
03:20:06 #25 [Debug] _4
03:20:06 #26 [Debug] _5
03:20:08 #27 [Warn] testPortOpen / message: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
129
FSharpOption<Unit>
      Value: <null>
